In [1]:
import duckdb
import pandas as pd
from sodapy import Socrata
import config

client = Socrata("data.ny.gov",
                  config.API_TOKEN,
                  username=config.NY_OPEN_DATA_USERNAME,
                  password=config.NY_OPEN_DATA_PASSWORD)

results = client.get("39hk-dx4f", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
%load_ext sql
conn = duckdb.connect(config.DEV_DATABASE)
%sql conn --alias duckdb


In [6]:
conn.sql("CREATE TABLE stations AS SELECT * FROM results_df")

CatalogException: Catalog Error: Table with name "stations" already exists!

In [3]:
conn.sql("select distinct line from stations")

┌────────────────────┐
│        line        │
│      varchar       │
├────────────────────┤
│ Crosstown          │
│ Dyre Av            │
│ Canarsie           │
│ Broadway - 7Av     │
│ Eastern Pky        │
│ Manhattan Bridge   │
│ West End           │
│ Staten Island      │
│ 8th Av - Fulton St │
│ Concourse          │
│    ·               │
│    ·               │
│    ·               │
│ Broadway           │
│ Jamaica            │
│ Franklin Shuttle   │
│ Queens Blvd        │
│ Jerome Av          │
│ Sea Beach          │
│ Myrtle Av          │
│ Queens - Archer    │
│ Nostrand           │
│ Second Av          │
├────────────────────┤
│ 34 rows (20 shown) │
└────────────────────┘

In [4]:
conn.close()

In [5]:
conn = duckdb.connect(config.DEV_DATABASE)

In [6]:
conn.sql("select * from stations")

┌──────────────┬────────────┬────────────┬──────────┬─────────────────────┬──────────────────────┬─────────┬─────────┬────────────────┬────────────┬───────────────┬────────────────┬───────────────────────┬───────────────────────┬─────────┬────────────────┬────────────────┬─────────────────────────────────────────────────────────┬─────────────────────────────┬─────────────────────────────┬─────────────────────────────┬─────────────┐
│ gtfs_stop_id │ station_id │ complex_id │ division │        line         │      stop_name       │ borough │   cbd   │ daytime_routes │ structure  │ gtfs_latitude │ gtfs_longitude │ north_direction_label │ south_direction_label │   ada   │ ada_northbound │ ada_southbound │                      georeference                       │ :@computed_region_yamh_8v7k │ :@computed_region_wbg7_3whc │ :@computed_region_kjdx_g34t │  ada_notes  │
│   varchar    │  varchar   │  varchar   │ varchar  │       varchar       │       varchar        │ varchar │ varchar │    varcha

In [7]:
conn.close()